# Alter data in CSV files

In [ ]:
import pandas as pd
import requests
import random
import time
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
cantons = ["zuerich", "bern", "luzern", "uri", "schwyz", "obwalden", "nidwalden", "glarus", "zug", "freiburg", 
           "solothurn", "basel-stadt", "basel-landschaft", "schaffhausen", "appenzell-ai", 
           "appenzell-ar", "st-gallen", "graubuenden", "aargau", "thurgau", "tessin", "waadt", "wallis", 
           "neuenburg", "genf", "jura"]

In [ ]:
canton = "solothurn"
print(canton)
filename = f"{canton}.csv"
df = pd.read_csv(filename)

# Extract the first column as a list
first_col = df.iloc[:, 0].tolist()

# Separate each value in the list using the specified rules
separated_values = []
for i, value in enumerate(first_col):
    #print(str(i))
    parts = [i]  # add index as the first entry
    current_part = ""
    if isinstance(value, str):
        for char in value:
            if char == ",":
                if current_part != "":
                    parts.append(int(current_part))
                    current_part = ""
            elif char == " ":
                continue
            else:
                current_part += char
        if current_part != "":
            parts.append(int(current_part))
        elif len(parts) < 3:  # add a 0 if the entry has no value
            parts.append(0)
        separated_values.append(parts)
    else: continue

# Check each row and modify the values as needed
for i, values in enumerate(separated_values):
    if len(values) >= 3 and values[2] < 10:
        combined_value = float(f"{values[1]}.{values[2]}")
        separated_values[i] = [values[0], combined_value] + values[3:]
    elif len(values) >= 2 and values[1] < 25:
        float_value = float(values[1])
        separated_values[i] = [values[0], float_value] + values[2:]

# Remove any row that has less than 4 entries
separated_values = [row for row in separated_values if len(row) >= 4]

new_df = pd.DataFrame(separated_values, columns=["Index", "Rooms", "Footage", "Rent"])
#new_df["Canton"] = canton  # add a column for the canton

new_col1 = df.loc[:,"header"]
new_col2 = df.loc[:,"header"]
new_col3 = df.loc[:,"header"]

# Insert the new columns at the beginning of the dataframe
df.insert(1, 'Rent', new_col3)
df.insert(1, 'Footage', new_col2)
df.insert(1, 'Rooms', new_col1)
df = df.drop("header",axis=1)

length = len(df.loc[:,'Rooms'])
for i in range(length):
    #print(str(i))
    if i in new_df['Index'].values:
        index = new_df.index[new_df['Index'] == i][0]
        #print(str(index))
        df.loc[i,"Rooms"]=new_df.loc[index,"Rooms"]
        df.loc[i,"Footage"]=new_df.loc[index,"Footage"]
        df.loc[i,"Rent"]=new_df.loc[index,"Rent"]
    else:
        df.loc[i,"Rooms"]=0
        df.loc[i,"Footage"]=0
        df.loc[i,"Rent"]=0

for i in range(length):
    if df.loc[i,"Rooms"]==0:
        df = df.drop(i)

df = df.reset_index(drop=True)
# Split the address column into street, city, and canton columns
df[['street', 'city', 'canton']] = df['address'].str.split(',', 2, expand=True)

# Strip whitespace from the new columns
df['street'] = df['street'].str.strip()
df['city'] = df['city'].str.strip()
df['canton'] = df['canton'].str.strip()

# Remove trailing "," from the new columns
df['street'] = df['street'].str.replace(r',$', '')
df['city'] = df['city'].str.replace(r',$', '')
df['canton'] = df['canton'].str.replace(r',$', '')

# Move data to the right if the canton column is empty
mask = df['canton'].isnull()
df.loc[mask, 'canton'] = df['city']
df.loc[mask, 'city'] = df['street']
df.loc[mask, 'street'] = ''
print(df)
filename = f"{canton}_new.csv"
df.to_csv(filename, index=False)

In [ ]:
df_combined = []
for canton in cantons:
    #canton = "nidwalden"
    print(canton)
    filename = f"{canton}.csv"
    df = pd.read_csv(filename)

    # Extract the first column as a list
    first_col = df.iloc[:, 0].tolist()

    # Separate each value in the list using the specified rules
    separated_values = []
    for i, value in enumerate(first_col):
        #print(str(i))
        parts = [i]  # add index as the first entry
        current_part = ""
        if isinstance(value, str):
            for char in value:
                if char == ",":
                    if current_part != "":
                        parts.append(int(current_part))
                        current_part = ""
                elif char == " ":
                    continue
                else:
                    current_part += char
            if current_part != "":
                parts.append(int(current_part))
            elif len(parts) < 3:  # add a 0 if the entry has no value
                parts.append(0)
            separated_values.append(parts)
        else: continue
            
    # Check each row and modify the values as needed
    for i, values in enumerate(separated_values):
        if len(values) >= 3 and values[2] < 10:
            combined_value = float(f"{values[1]}.{values[2]}")
            separated_values[i] = [values[0], combined_value] + values[3:]
        elif len(values) >= 2 and values[1] < 25:
            float_value = float(values[1])
            separated_values[i] = [values[0], float_value] + values[2:]

    # Remove any row that has less than 4 entries
    separated_values = [row for row in separated_values if len(row) >= 4]

    new_df = pd.DataFrame(separated_values, columns=["Index", "Rooms", "Footage", "Rent"])
    #new_df["Canton"] = canton  # add a column for the canton

    new_col1 = df.loc[:,"header"]
    new_col2 = df.loc[:,"header"]
    new_col3 = df.loc[:,"header"]

    # Insert the new columns at the beginning of the dataframe
    df.insert(1, 'Rent', new_col3)
    df.insert(1, 'Footage', new_col2)
    df.insert(1, 'Rooms', new_col1)
    df = df.drop("header",axis=1)

    length = len(df.loc[:,'Rooms'])
    for i in range(length):
        #print(str(i))
        if i in new_df['Index'].values:
            index = new_df.index[new_df['Index'] == i][0]
            #print(str(index))
            df.loc[i,"Rooms"]=new_df.loc[index,"Rooms"]
            df.loc[i,"Footage"]=new_df.loc[index,"Footage"]
            df.loc[i,"Rent"]=new_df.loc[index,"Rent"]
        else:
            df.loc[i,"Rooms"]=0
            df.loc[i,"Footage"]=0
            df.loc[i,"Rent"]=0

    for i in range(length):
        if df.loc[i,"Rooms"]==0:
            df = df.drop(i)

    df = df.reset_index(drop=True)
    # Split the address column into street, city, and canton columns
    df[['street', 'city', 'canton']] = df['address'].str.split(',', 2, expand=True)

    # Strip whitespace from the new columns
    df['street'] = df['street'].str.strip()
    df['city'] = df['city'].str.strip()
    df['canton'] = df['canton'].str.strip()

    # Remove trailing "," from the new columns
    df['street'] = df['street'].str.replace(r',$', '')
    df['city'] = df['city'].str.replace(r',$', '')
    df['canton'] = df['canton'].str.replace(r',$', '')

    # Move data to the right if the canton column is empty
    mask = df['canton'].isnull()
    df.loc[mask, 'canton'] = df['city']
    df.loc[mask, 'city'] = df['street']
    df.loc[mask, 'street'] = ''
    
    df_combined.append(df)
    #print(df)
# Concatenate all the DataFrames into a single DataFrame
df_combined_swiss = pd.concat(df_combined, ignore_index=True)
df_combined_swiss.to_csv("swiss.csv", index=False)
 

In [ ]:
# Set the base URL for the Overpass API
base_url = "https://nominatim.openstreetmap.org/search"
headers = headers = ({'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'})

In [ ]:
cantons = ["zuerich", "bern", "luzern", "uri", "schwyz", "obwalden", "nidwalden", "glarus", "zug", "freiburg", 
           "solothurn", "basel-stadt", "basel-landschaft", "schaffhausen", "appenzell-ai", 
           "appenzell-ar", "st-gallen", "graubuenden", "aargau", "thurgau", "tessin", "waadt", "wallis", 
           "neuenburg", "genf", "jura"]

In [ ]:
# Function to get latitude and longitude for a given address
def get_lat_lng(address):
    try:
        params = {"q": address, "format": "json"}
        response = requests.get(base_url, params=params, headers=headers)
        data = response.json()
        if data:
            lat = float(data[0]['lat'])
            lng = float(data[0]['lon'])
            return lat, lng
        else:
            return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None

In [ ]:
def get_lat_lng_from_city(city_postalcode):
    try:
        city, postalcode = city_postalcode.split()
        params = {"q": f"{city} {postalcode}", "format": "json"}
        response = requests.get(base_url, params=params, headers=headers)
        data = response.json()
        if data:
            lat = float(data[0]['lat'])
            lng = float(data[0]['lon'])
            return lat, lng
        else:
            return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None

In [ ]:
# Add new columns for latitude and longitude
df["Latitude"] = None
df["Longitude"] = None

In [ ]:
# Iterate through the dataframe and update the latitude and longitude columns
def update_lat_lng(df):
    # Create columns for the address and city coordinates
    df['Address_Latitude'] = ''
    df['Address_Longitude'] = ''
    df['City_Latitude'] = ''
    df['City_Longitude'] = ''

    # Iterate through the dataframe and update the latitude and longitude columns
    for index, row in df.iterrows():
        address = row["address"]
        city = row["city"]
        address_lat, address_lng = get_lat_lng(address)
        time.sleep(1)
        city_lat, city_lng = get_lat_lng(city)
        df.at[index, "Address_Latitude"] = address_lat
        df.at[index, "Address_Longitude"] = address_lng
        df.at[index, "City_Latitude"] = city_lat
        df.at[index, "City_Longitude"] = city_lng
        if index % 100 == 0:
            print(f"Processed {index} rows")
        time.sleep(1)


In [ ]:
filename = "swiss.csv"
df = pd.read_csv(filename)
update_lat_lng(df)

In [ ]:
# Save the updated dataframe to a new CSV file
df.to_csv("swiss_with_coordinates.csv", index=False)

In [ ]:
# Read the CSV file
df = pd.read_csv("swiss_with_coordinates.csv")

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
df['Distance_to_City_Center(km)'] = df.apply(
    lambda row: haversine(
        row['Address_Latitude'], row['Address_Longitude'],
        row['City_Latitude'], row['City_Longitude']
    ), axis=1
)


In [ ]:
# Drop rows with missing values in the specified columns
df = df.dropna(subset=["Rooms", "Footage", "Rent", "address","Distance_to_City_Center(km)","Address_Latitude","Address_Longitude","City_Latitude","City_Longitude"])
# Drop duplicates
df = df.drop_duplicates()

# Function to remove outliers using the IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Remove outliers in the specified columns
df = remove_outliers(df, "Rooms")
df = remove_outliers(df, "Footage")
df = remove_outliers(df, "Rent")

# Save the dataframe without outliers to a new CSV file
df.to_csv("swiss_no_outliers_and_cleaned.csv", index=False)

In [ ]:
#Visualize the distribution of the "Rooms", "Footage", and "Rent" columns using histograms:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df["Rooms"], kde=True, bins=20)
plt.title("Rooms Distribution")

plt.subplot(1, 3, 2)
sns.histplot(df["Footage"], kde=True, bins=20)
plt.title("Footage Distribution")

plt.subplot(1, 3, 3)
sns.histplot(df["Rent"], kde=True, bins=20)
plt.title("Rent Distribution")

plt.show()

In [ ]:
#Visualize the relationship between "Footage" and "Rent" using a scatterplot:
plt.figure(figsize=(10, 6))
sns.scatterplot(x="Footage", y="Rent", data=df, hue="Rooms")
plt.title("Footage vs. Rent")
plt.show()

In [ ]:
#Visualize the relationship between "Rooms" and "Rent" using a boxplot:
plt.figure(figsize=(10, 6))
sns.boxplot(x="Rooms", y="Rent", data=df)
plt.title("Rooms vs. Rent")
plt.show()

In [ ]:
#Create a heatmap to visualize the correlation between numeric columns:
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()